# NER Fine Tuning with Hugging Face

This notebook evaluates using Hugging Face's NER pipeline for fine tuning on the labelled articles. The Specter2 model is intended to be used as a BERT based encoder model with a token classification head.

Specter2 model card on HF hub: https://huggingface.co/allenai/specter2

In [36]:
import os, sys

import pandas as pd
import numpy as np
from datasets import load_dataset

from transformers import AutoTokenizer

# use autoreload
%load_ext autoreload
%autoreload 2

In [3]:
wnut = load_dataset("wnut_17")

Found cached dataset wnut_17 (C:/Users/tyand/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)
100%|██████████| 3/3 [00:00<00:00, 332.88it/s]


In [31]:
# ensure src is in path
sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))

In [41]:
from src.entity_extraction.entity_extraction_evaluation import get_token_labels, load_json_label_files

labelled_file_path = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir, "data", "labelled", "labelled")

In [42]:
all_text, labelled_entities = load_json_label_files(labelled_file_path)

In [48]:
tokens, token_labels = get_token_labels(labelled_entities, all_text)

In [50]:
id2label = {
    0: "O",
    1: "B-GEOG",
    2: "I-GEOG",
    3: "B-SITE",
    4: "I-SITE",
    5: "B-EMAIL",
    6: "I-EMAIL",
    7: "B-ALTI",
    8: "I-ALTI",
    9: "B-TAXA",
    10: "I-TAXA",
    11: "B-REGION",
    12: "I-REGION",
    13: "B-AGE",
    14: "I-AGE",
}

label2id = {
    "O": 0,
    "B-GEOG": 1,
    "I-GEOG": 2,
    "B-SITE": 3,
    "I-SITE": 4,
    "B-EMAIL": 5,
    "I-EMAIL": 6,
    "B-ALTI": 7,
    "I-ALTI": 8,
    "B-TAXA": 9,
    "I-TAXA": 10,
    "B-REGION": 11,
    "I-REGION": 12,
    "B-AGE": 13,
    "I-AGE": 14,
}

In [51]:
#  convert the labels to ids
token_label_ids = [label2id[label] for label in token_labels]

In [90]:
# split the data into chunks of 128 tokens and labels
chunked_tokens = [tokens[i : i + 128] for i in range(0, len(tokens), 128)]
chunked_token_label_ids = [token_label_ids[i : i + 128] for i in range(0, len(token_label_ids), 128)]
chunked_labels = [token_labels[i : i + 128] for i in range(0, len(token_labels), 128)]

# make each chunk a dict with keys ner_tags and tokens
chunked_data = [
    {
        "ner_tags": chunked_token_label_ids[i], 
        "tokens": chunked_tokens[i],
        "labels": chunked_labels[i]
     } for i in range(len(chunked_tokens))
]
# make the data into a huggingface dataset
dataset = pd.DataFrame(chunked_data)

from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(dataset)

In [70]:
# needed as tokenizing adds CLS and SEP tokens, doesn't match labels
# see here for more detail: https://huggingface.co/docs/transformers/tasks/token_classification
# It does:
# 1. Mapping all tokens to their corresponding word with the word_ids method.
# 2. Assigning the label -100 to the special tokens [CLS] and [SEP] so they’re ignored by the PyTorch loss function.
# 3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [71]:
tokenizer = AutoTokenizer.from_pretrained("allenai/specter2")

tokenized = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/833 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [67]:
len(tokenized["labels"][0])

184

## Split into Train/Test

In [81]:
from sklearn.model_selection import train_test_split    

train, test = train_test_split(tokenized, test_size=0.2, random_state=42)

In [95]:
# create a new huggingface daatset with train/test
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)

dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset})

In [96]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 666
    })
    test: Dataset({
        features: ['ner_tags', 'tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 167
    })
})

In [85]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [86]:
import evaluate

seqeval = evaluate.load("seqeval")

In [78]:
import numpy as np

label_list = label2id.keys()

# labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [79]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "allenai/specter2", num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/specter2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
# downgraded to transformers==4.28.0 due to PartialState import error, referenced here and was recommended solution: 
# https://github.com/huggingface/transformers/issues/22816 

training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    no_cuda=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()